LANCHAIN Memory

Conversation BufferMemory

In [ ]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory



The components of the conversation. ConversationBufferMemory ( k=#) .  

In [ ]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True  #False is to the point
)

In [ ]:
conversation.predict(input="Hi, my name is Edilma")

Verbose false gave me "Hello Edilma, it's nice to meet you! I am an AI language model designed to have conversations with humans. How can I assist you today?"
Verbose True gave me > 
Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Edilma
AI:

> Finished chain.
"Hello Edilma, it's nice to meet you! I am an AI language model designed to have conversations with humans. How can I assist you today?"

In [ ]:
conversation.predict(input="What is 1+1?")

In [ ]:
conversation.predict(input="What is my name?")


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Edilma
AI: Hello Edilma, it's nice to meet you! I am an AI language model designed to have conversations with humans. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.
'Your name is Edilma, as you mentioned earlier.'

#To check what we have in memory until now 

In [ ]:
print(memory.buffer) 

In [ ]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Edilma\nAI: Hello Edilma, it's nice to meet you! I am an AI language model designed to have conversations with humans. How can I assist you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2.\nHuman: What is my name?\nAI: Your name is Edilma, as you mentioned earlier."}

In [1]:
memory = ConversationBufferMemory()  

NameError: name 'ConversationBufferMemory' is not defined

The chatbox appears to have memory by providing the full conversation in "context". LLM are stateless and each transaction is independent

In [ ]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})
print(memory.buffer)


Human: Hi
AI: What's up

In [ ]:
memory.load_memory_variables({})


{'history': "Human: Hi\nAI: What's up"}

The longer the conversation the more tokens we use... Lang

# ConversationBufferWindowMemory 

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

K=1 means save one conversation for the human and one for the chat.  It keeps the most k recent interaction.  

In [ ]:
memory = ConversationBufferWindowMemory(k=1)    

In [ ]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)



In the caso anterior it won't remember my name fter 1+1=2 because it has kept in memory only one interaction... 

# CONVERSATION TokenBufferMemory
# pip install tiktoken

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [ ]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

t keeps in memory the most recent exchanges up to  max_token_limit=#

In [ ]:
memory.load_memory_variables({})

this is 50 tokens
{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

The llm creates a sumary and the system try to keep the explicit storage up to the number of token that we
specified .  Now we we save it in memory now 